**Importing of necessary modules**


In [1]:
#base64 module is used to encode and decode data. 
#we have considered already exisitng datasets present in the sklearn library
import pandas as pd
import numpy as np
import base64
import plotly.graph_objects as graph
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.datasets import load_breast_cancer

**Importing StreamLit Library**

In [2]:
#install stramlit using pip install command
!pip install streamlit --quiet

     |████████████████████████████████| 9.1 MB 5.6 MB/s 
     |████████████████████████████████| 111 kB 46.7 MB/s 
     |████████████████████████████████| 164 kB 51.6 MB/s 
     |████████████████████████████████| 232 kB 21.6 MB/s 
     |████████████████████████████████| 4.3 MB 38.3 MB/s 
     |████████████████████████████████| 77 kB 2.0 MB/s 
     |████████████████████████████████| 181 kB 48.4 MB/s 
     |████████████████████████████████| 63 kB 608 kB/s 
     |████████████████████████████████| 131 kB 45.0 MB/s 
     |████████████████████████████████| 428 kB 44.6 MB/s 
     |████████████████████████████████| 130 kB 46.1 MB/s 
     |████████████████████████████████| 793 kB 43.0 MB/s 
     |████████████████████████████████| 381 kB 41.3 MB/s 
     |████████████████████████████████| 51 kB 2.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.4 requi

In [1]:
import streamlit as st

2022-06-05 07:54:02.769 INFO    numexpr.utils: NumExpr defaulting to 2 threads.


**Creation of app.py file**

We have considered building of an HyperParameter-Optimizer App to generate the best paramters for the model.


We have Made use of the RandomForestRegressor to build a model wherein the user is expected to upload a csv file(datasets).

After Uploading the csv file , user is also expected to set intial values for various Learning Parameter over here,based on which the model is trained and provides us with the accurate values for various parameters to optimize the model to render a better accuracy for several predicitons.

In [2]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import base64
import plotly.graph_objects as graph
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.datasets import load_breast_cancer

st.set_page_config(page_title='''Machine Learning HyperParameter Optimizer App''',layout='wide')
st.header('''Machine Learning HyperParameter Optimizer App''')
st.write('''We have made use of RandomForestRegressor() to build a regression model using Random Forest algorithm''')
#creates a sidebar along with a respective header
st.sidebar.header("Upload the csv file(dataset file)")
#upload_file=st.sidebar.file_uploader("Upload your csv file")
uploaded_file = st.sidebar.file_uploader("Upload your input CSV file", type=["csv"])
st.sidebar.markdown("""
[Example CSV input file](https://raw.githubusercontent.com/dataprofessor/data/master/delaney_solubility_with_descriptors.csv)
""")

#now consider the user defined inputs for the parameters.
#now we have considered spliting up of datasets into Training and Testing Data
st.sidebar.header('Set Parameters')
split_size=st.sidebar.slider('Data split ratio(Spliting of data into the datasets)')

#LEARNING PARAMETERS
#Now we need to define a Learning Parameter for defining a perfect model which can be utilized to arrive at better accuracy
parameter_n_estimators=st.sidebar.slider('Number of estimators',0,500,(10,50),50)

#initialy we have considered number of esitmators to range from 0 to 500
#here the step size varing size is considered to be 50
parameter_n_estimators_stepsize=st.sidebar.number_input('Step Size is defined for Number of estimators',10)

#now we have consider another parameter which is max_features
parameter_max_features = st.sidebar.slider('Max features (max_features)', 1, 50, (1,3), 1)
st.sidebar.number_input('Step size for max_features', 1)

#paramters considered are min_samples_split along with min_sample_leaf
parameter_min_samples_split = st.sidebar.slider('Minimum number of samples required to split an internal node (min_samples_split)', 1, 10, 2, 1)
parameter_min_samples_leaf = st.sidebar.slider('Minimum number of samples required to be at a leaf node (min_samples_leaf)', 1, 10, 2, 1)

#GENERAL PARAMETERS
#now are considering the general parameters
st.sidebar.subheader('General Parameters')
#random state
parameter_random_state = st.sidebar.slider('Seed number (random_state)', 0, 1000, 42, 1)
#criterion
parameter_criterion = st.sidebar.select_slider('Performance measure (criterion)', options=['mse', 'mae'])
#bootstrap
parameter_bootstrap = st.sidebar.select_slider('Bootstrap samples when building trees (bootstrap)', options=[True, False])
#oob_score
parameter_oob_score = st.sidebar.select_slider('Whether to use out-of-bag samples to estimate the R^2 on unseen data (oob_score)', options=[False, True])
#n_jobs
parameter_n_jobs = st.sidebar.select_slider('Number of jobs to run in parallel (n_jobs)', options=[1, -1])

n_estimators_range = np.arange(parameter_n_estimators[0], parameter_n_estimators[1]+parameter_n_estimators_stepsize, parameter_n_estimators_stepsize)
max_features_range = np.arange(parameter_max_features[0], parameter_max_features[1]+1, 1)
param_grid = dict(max_features=max_features_range, n_estimators=n_estimators_range) 

st.subheader('Dataset')

def build_model(df):
    X = df.iloc[:,:-1] 
    # Using all column except for the last column as X(here the last column is considered to be the output column(target))
    Y = df.iloc[:,-1]
    # We will be considering it as the target over here

    st.markdown('A model is being built to predict the following **Y** variable:')
    st.info(Y.name)

    # Data splitting takes place where in we split the data into training and testing data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=split_size)

    #we have considered a RandomForestRegressor model
    # here the inputs for the parameters are basically provided by the users.
    rf_model = RandomForestRegressor(n_estimators=parameter_n_estimators,random_state=parameter_random_state,max_features=parameter_max_features,
        criterion=parameter_criterion,
        min_samples_split=parameter_min_samples_split,
        min_samples_leaf=parameter_min_samples_leaf,
        bootstrap=parameter_bootstrap,
        oob_score=parameter_oob_score,
        n_jobs=parameter_n_jobs)

    #here we have made use of GridSearchCV
    #GridSearchCV basically finds the best parameters for the model which inturn provides best accuracy for the model.
    grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5)
    grid.fit(X_train, Y_train)

    #now we will consider the performance of the model
    st.subheader('Performance of the Model')
    Y_pred_test = grid.predict(X_test)
    #we will basically generate the result for r2score along with mae(mean_accuracy_error)
    st.write('Coefficient of determination ($R^2$):')
    st.info( r2_score(Y_test, Y_pred_test) )
    st.write('Error (MSE or MAE):')
    st.info( mean_squared_error(Y_test, Y_pred_test) )

    st.write(f"The best parameters are ${grid.best_params_} with a score of ${grid.best_score_}")

    st.subheader('Model Parameters')
    st.write(grid.get_params())

    #-----Process grid data-----#
    grid_results = pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["R2"])],axis=1)
    # Segment data into groups based on the 2 hyperparameters
    grid_contour = grid_results.groupby(['max_features','n_estimators']).mean()
    # Pivoting the data
    grid_reset = grid_contour.reset_index()
    grid_reset.columns = ['max_features', 'n_estimators', 'R2']
    grid_pivot = grid_reset.pivot('max_features', 'n_estimators')
    x = grid_pivot.columns.levels[1].values
    y = grid_pivot.index.values
    z = grid_pivot.values

    #-----Plot-----#
    layout = graph.Layout(
            xaxis=graph.layout.XAxis(
              title=graph.layout.xaxis.Title(
              text='n_estimators')
             ),
             yaxis=graph.layout.YAxis(
              title=graph.layout.yaxis.Title(
              text='max_features')
            ) )
    fig = graph.Figure(data= [graph.Surface(z=z, y=y, x=x)], layout=layout )
    fig.update_layout(title='Hyperparameter tuning',
                      scene = dict(
                        xaxis_title='n_estimators',
                        yaxis_title='max_features',
                        zaxis_title='R2'),
                      autosize=False,
                      width=800, height=800,
                      margin=dict(l=65, r=50, b=65, t=90))
    st.plotly_chart(fig)
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write(df)
    build_model(df)

Writing app.py


In [3]:
!streamlit run app.py & npx localtunnel --port 8501

2022-06-05 07:54:28.021 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.74.104.249:8501

npx: installed 22 in 6.999s
your url is: https://early-hornets-work-34-74-104-249.loca.lt
2022-06-05 07:55:29.543 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/scriptrunner/script_runner.py", line 554, in _run_script
    exec(code, module.__dict__)
  File "app.py", line 140, in <module>
    build_model(df)
  File "app.py", line 74, in build_model
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=split_size)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py", line 2421, in train_test_split
    n_samples, test_size, train_size, default_test_size=0.25
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py", line 2046, in _validat